# Implementering av et dyreekspertsystem

Et eksempel fra [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

I dette eksemplet skal vi implementere et enkelt kunnskapsbasert system for å bestemme et dyr basert på noen fysiske kjennetegn. Systemet kan representeres ved følgende AND-OR-tre (dette er en del av hele treet, vi kan enkelt legge til flere regler):

![](../../../../../../translated_images/no/AND-OR-Tree.5592d2c70187f283.webp)


## Vårt eget ekspertsystemskall med bakoverinferenz

La oss prøve å definere et enkelt språk for kunnskapsrepresentasjon basert på produksjonsregler. Vi vil bruke Python-klasser som nøkkelord for å definere regler. Det vil i hovedsak være 3 typer klasser:
* `Ask` representerer et spørsmål som må stilles til brukeren. Den inneholder settet av mulige svar.
* `If` representerer en regel, og det er bare syntaktisk sukker for å lagre innholdet i regelen
* `AND`/`OR` er klasser for å representere AND/OR-grener i treet. De lagrer bare listen av argumenter inne. For å forenkle koden, er all funksjonalitet definert i overklasse `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

I vårt system ville arbeidsminnet inneholde listen over **fakta** som **attributt-verdi-par**. Kunnskapsbasen kan defineres som en stor ordbok som kobler handlinger (nye fakta som skal settes inn i arbeidsminnet) til betingelser, uttrykt som OG-ELLER-uttrykk. Også noen fakta kan `Ask`es.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

For å utføre bakoverinference vil vi definere klassen `Knowledgebase`. Den vil inneholde:
* Arbeidende `memory` - et oppslagsord som kobler attributter til verdier
* Kunnskapsbasens `rules` i det formatet som er definert ovenfor

To hovedmetoder er:
* `get` for å hente verdien av et attributt, og utføre inferens om nødvendig. For eksempel vil `get('color')` hente verdien for en fargeplass (den vil spørre om nødvendig, og lagre verdien for senere bruk i arbeidsminnet). Hvis vi spør `get('color:blue')`, vil den spørre etter en farge, for så å returnere `y`/`n` avhengig av fargen.
* `eval` utfører selve inferensen, altså går gjennom AND/OR-treet, evaluerer under-mål, osv.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

La oss nå definere vårt dyrekunnskapsbase og utføre konsultasjonen. Vær oppmerksom på at dette oppkallet vil stille deg spørsmål. Du kan svare ved å skrive `y`/`n` for ja/nei-spørsmål, eller ved å angi tall (0..N) for spørsmål med lengre flervalgssvar.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Bruke Experta for framoverinferenz

I det neste eksemplet vil vi prøve å implementere framoverinferenz ved hjelp av et av bibliotekene for kunnskapsrepresentasjon, [Experta](https://github.com/nilp0inter/experta). **Experta** er et bibliotek for å lage framoverinferenzsystemer i Python, som er designet for å være likt det klassiske gamle systemet [CLIPS](http://www.clipsrules.net/index.html).

Vi kunne også ha implementert framoverkobling selv uten mange problemer, men naive implementasjoner er vanligvis ikke veldig effektive. For mer effektiv regelmatching brukes en spesiell algoritme [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Vi vil definere systemet vårt som en klasse som underklasser `KnowledgeEngine`. Hver regel defineres av en egen funksjon med `@Rule`-annotasjon, som spesifiserer når regelen skal utløses. Inne i regelen kan vi legge til nye fakta ved hjelp av `declare`-funksjonen, og ved å legge til disse faktaene vil flere regler bli kalt av den fremadrettede inferensmotoren.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Når vi har definert en kunnskapsbase, fyller vi vår arbeidsminne med noen innledende fakta, og deretter kaller vi `run()`-metoden for å utføre inferensen. Du kan se som et resultat at nye utledede fakta legges til i arbeidsminnet, inkludert det endelige faktumet om dyret (hvis vi setter opp alle de innledende faktaene riktig).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokumentet er oversatt ved bruk av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vennligst vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på dets opprinnelige språk skal betraktes som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi påtar oss ikke ansvar for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
